# 2nd Level Model Structure: Reboiler

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from pyomo import environ as pe
from global_sets.component import m
from utility.display_utility import trans_product_mole, trans_product_mass
from utility.model_utility import add_dual, update_dual, check_DOF

# stage construction rules
from physics.energy_reboiler import energy_block_rule
from physics.VLE_reboiler_MPCC_P import VLE_block_rule
from physics.MPCC_P import P_NCP_block_rule, P_Reg_block_rule, P_pf_block_rule

model = pe.ConcreteModel()

# Global Sets (Inlet / Outlet)

In [3]:
model.inlet = pe.Set(initialize=['in'])
model.outlet = pe.Set(initialize=['out','P'])
model.stream = model.inlet | model.outlet

# Global Variables

In [4]:
# Tray Inlet/Outlet Variable
model.x_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.x = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.z = pe.Var(m.COMP_FEED,within=pe.NonNegativeReals)

model.L = pe.Var(model.stream,within=pe.NonNegativeReals)
model.V = pe.Var(model.stream,within=pe.NonNegativeReals)
model.F = pe.Var(within=pe.NonNegativeReals)

model.H_L_ = pe.Var(model.inlet,within=pe.Reals)
model.H_V_ = pe.Var(model.inlet,within=pe.Reals)
model.H_L = pe.Var(within=pe.Reals)
model.H_V = pe.Var(within=pe.Reals)
model.H_F = pe.Var(within=pe.Reals)

# State Variable
model.T = pe.Var(within=pe.NonNegativeReals,bounds=(200+273.15,350+273.15)) # K
model.T_F = pe.Var(within=pe.NonNegativeReals) # K
model.P = pe.Var(within=pe.NonNegativeReals,bounds=(10,30)) # Bar

model.f_V = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)
model.f_L = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)

model.Q_main = pe.Var(within=pe.Reals) # MW

# Construct Individual Blocks

In [5]:
model.energy_block = pe.Block(rule=energy_block_rule)

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| energy_block.dH_F
| energy_block.dH_V
| energy_block.dH_L
| energy_block.dH_vap
--------------------------------------------------



In [6]:
model.VLE_block = pe.Block(rule=VLE_block_rule)

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| VLE_block.P_VLE
| VLE_block.n_ave
| VLE_block.n_ave_cal
| VLE_block.Hen
| VLE_block.Hen0
| VLE_block.gamma
| VLE_block.P_sat
| VLE_block.P_sat_Y
| VLE_block.P_sat_dY_inf
| VLE_block.P_sat_dY0
| VLE_block.Hen_ref
| VLE_block.Hen0_ref
| VLE_block.gamma_ref
| VLE_block.V_L
| VLE_block.V_L_dY_inf
| VLE_block.V_L_dY0
| VLE_block.poynting
--------------------------------------------------



# Standard MESH Equations

## Mass Balance

\begin{equation}
Fz_{i}+\sum_{s\in{inlet}}L_{s}x'_{i,s}+\sum_{s\in{inlet}}V_{s}y'_{i,s}-\sum_{s\in{outlet}}L_{s}x_{i,s}+\sum_{s\in{ouelet}}V_{s}y_{i,s}+R_{i} = 0 \\
i = 1,...NC
\end{equation}

In [7]:
def mass_balance_main_rule(model,i):
    if i in m.COMP_FEED:
        return model.F*model.z[i] + sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
    else:
        return sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
model.mass_balance_main_con = pe.Constraint(m.COMP_TOTAL,rule=mass_balance_main_rule)

## Equilibrium

\begin{align}
&f_{i,V} = f_{i,L} \\
&i = 1,...NC
\end{align}

In [8]:
def VL_equil_rule(model,i):
    return model.f_V[i] == model.f_L[i]
model.VL_equil_con = pe.Constraint(m.COMP_TOTAL,rule=VL_equil_rule)

# MPCC - P-Pspec

In [9]:
model.MPCC = pe.Block(rule = P_Reg_block_rule)

> Importing MPCC_P_Reg Blocks......
> Adding the following local variable:
--------------------------------------------------
| MPCC.s_L
| MPCC.s_V
| MPCC.epi
--------------------------------------------------
> Adding complementarity constraint, spliting pressure used in VLE



## Summation

\begin{equation}
\sum_ix_{i} = \sum_iy_{i} \\
F + L_{in} + V_{in} + R = L_{out} + V_{out}
\end{equation}

In [10]:
def summation_x_y_rule(model):
    return sum(model.x[i] for i in m.COMP_TOTAL) == sum(model.y[i] for i in m.COMP_TOTAL)
model.summation_x_y_con = pe.Constraint(rule=summation_x_y_rule)

def summation_total_mass_rule(model):
    return model.F + sum(model.L[s] + model.V[s] for s in model.inlet)\
            - sum(model.L[s] + model.V[s] for s in model.outlet) == 0
model.summation_total_mass_con = pe.Constraint(rule=summation_total_mass_rule)

## Energy Balance

\begin{equation}
F H_f+\sum_{s\in{inlet}}L_{s}H_{l,s}+\sum_{s\in{inlet}}V_{s}H_{v,s}-\sum_{s\in{outlet}}L_{s}H_{l,s}-\sum_{s\in{outlet}}V_{s}H_{v,s}+Q = 0
\end{equation}

In [11]:
def heat_balance_main_rule(model):
    return model.F*model.H_F + sum(model.L[s]*model.H_L_[s] + model.V[s]*model.H_V_[s] for s in model.inlet) \
            + model.Q_main - sum(model.L[s]*model.H_L + model.V[s]*model.H_V for s in model.outlet) == 0
model.heat_balance_main_con = pe.Constraint(rule=heat_balance_main_rule)

# Testing

In [12]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


# Load a flash liquid composition as feed

In [13]:
with open('../saved_solutions/reactive_flash_MPCC_P_pf_200C.pickle', 'rb') as f:
    C200 = pickle.load(f)

## Fixing In/Product Redundent Flow

In [14]:
model.y_.fix(0)
for i in m.COMP_TOTAL:
    model.x_['in',i].fix(C200.Solution.Variable['x[{}]'.format(i)]['Value'])
model.L['in'].fix(1)
model.L['P'].fix(0)
model.V['in'].fix(0)
model.V['P'].fix(0)
model.H_V_.fix(0)
model.H_L_.fix(C200.Solution.Variable['H_L']['Value'])

## Fixing Model Parameters

In [15]:
model.P.fix(20)
model.T_F.fix(200+273.15)
model.F.fix(0)
model.z.fix(0)
model.VLE_block.n_ave.fix(20)
model.Q_main.fix(0)

In [16]:
check_DOF(pe,model)

Active Equality Constraints:	 957
Active Inequality Constraints:	 2
Active Variables:		 1131
Fixed Variables:		 172
DOF:				 2


## Fix T or to fix Q? we have found that Maximize against an upper bound is most reliable

In [17]:
model.T.setub(300+273.15)

In [18]:
# model.obj = pe.Objective(expr = model.T ,sense=pe.maximize)
# model.obj = pe.Objective(expr = model.MPCC.pf,sense=pe.minimize)
# model.obj = pe.Objective(expr = model.T - model.MPCC.pf ,sense=pe.maximize)

In [19]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'

opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'
# opt.options['recalc_y'] = 'yes'
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling =mc19
linear_scaling_on_demand =no
max_iter=7000


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with

  77  0.0000000e+00 1.48e+02 1.11e+03  -1.0 4.44e+03    -  6.97e-03 3.11e-02w  1
  78  0.0000000e+00 1.48e+02 6.31e+04  -1.0 4.28e+03    -  2.63e-02 3.56e-04w  1
  79  0.0000000e+00 1.07e+00 2.52e+03  -1.0 2.38e+03    -  8.52e-01 1.67e-03h  8
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  0.0000000e+00 1.07e+00 2.92e+03  -1.0 7.29e+03    -  1.96e-01 1.67e-03h  9
  81  0.0000000e+00 1.07e+00 5.06e+03  -1.0 7.27e+03    -  8.94e-01 1.67e-03h  9
  82  0.0000000e+00 1.07e+00 5.85e+03  -1.0 7.26e+03    -  1.99e-01 1.67e-03h  9
  83  0.0000000e+00 1.07e+00 1.05e+04  -1.0 7.25e+03    -  1.00e+00 1.67e-03h  9
  84  0.0000000e+00 1.07e+00 1.21e+04  -1.0 7.24e+03    -  1.98e-01 1.67e-03h  9
  85  0.0000000e+00 1.07e+00 2.14e+04  -1.0 7.23e+03    -  1.00e+00 1.67e-03h  9
  86  0.0000000e+00 1.07e+00 2.47e+04  -1.0 7.22e+03    -  2.03e-01 1.67e-03h  9
  87  0.0000000e+00 1.07e+00 4.95e+04  -1.0 7.21e+03    -  1.00e+00 1.68e-03h  9
  88  0.0000000e+00 1.07e+00

 172  0.0000000e+00 1.29e+00 2.39e+12  -1.0 6.03e+03    -  1.00e+00 3.67e-03h  8
 173  0.0000000e+00 1.30e+00 2.39e+12  -1.0 6.01e+03    -  4.68e-01 3.68e-03h  8
 174  0.0000000e+00 1.30e+00 2.39e+12  -1.0 5.99e+03    -  1.00e+00 3.68e-03h  8
 175  0.0000000e+00 1.30e+00 2.39e+12  -1.0 5.97e+03    -  4.78e-01 3.69e-03h  8
 176  0.0000000e+00 1.31e+00 2.50e+12  -1.0 5.95e+03    -  1.00e+00 3.69e-03h  8
 177  0.0000000e+00 1.31e+00 2.50e+12  -1.0 5.93e+03    -  4.89e-01 3.70e-03h  8
 178  0.0000000e+00 1.31e+00 2.51e+12  -1.0 5.91e+03    -  1.00e+00 3.71e-03h  8
 179  0.0000000e+00 1.32e+00 2.51e+12  -1.0 5.89e+03    -  5.00e-01 3.71e-03h  8
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 180  0.0000000e+00 1.32e+02 1.68e+12  -1.0 5.87e+03    -  1.00e+00 4.76e-01w  1
 181  0.0000000e+00 1.27e+02 1.58e+12  -1.0 3.22e+03    -  1.27e-02 4.32e-02w  1
 182  0.0000000e+00 1.27e+02 6.84e+13  -1.0 3.08e+03    -  7.31e-02 5.09e-04w  1
 183  0.0000000e+00 1.32e+00

 273  0.0000000e+00 1.03e+02 6.20e+14  -1.0 2.07e+03    -  1.00e+00 8.82e-04w  1
 274  0.0000000e+00 1.44e+00 3.22e+12  -1.0 2.06e+03    -  3.08e-02 1.05e-03h  9
 275  0.0000000e+00 1.44e+00 3.22e+12  -1.0 4.57e+03    -  8.61e-01 1.05e-03h 10
 276  0.0000000e+00 1.44e+00 3.22e+12  -1.0 4.56e+03    -  2.69e-02 1.05e-03h 10
 277  0.0000000e+00 1.44e+00 3.22e+12  -1.0 4.55e+03    -  1.00e+00 9.46e-04h 10
 278  0.0000000e+00 1.44e+00 3.22e+12  -1.0 4.54e+03    -  2.30e-02 8.46e-04h 10
 279  0.0000000e+00 1.43e+00 3.22e+12  -1.0 4.54e+03    -  2.60e-01 7.61e-04h 10
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 280  0.0000000e+00 1.43e+00 3.22e+12  -1.0 4.53e+03    -  2.21e-02 6.89e-04h 10
 281  0.0000000e+00 1.43e+00 3.22e+12  -1.0 4.53e+03    -  1.00e+00 6.27e-04h 10
 282  0.0000000e+00 1.43e+00 3.22e+12  -1.0 4.52e+03    -  1.79e-02 5.75e-04h 10
 283  0.0000000e+00 1.43e+00 3.22e+12  -1.0 4.51e+03    -  2.55e-01 5.29e-04h 10
 284  0.0000000e+00 2.44e+01

 366  0.0000000e+00 1.39e+00 3.32e+12  -1.0 4.00e+03    -  1.87e-02 5.38e-04h 10
 367  0.0000000e+00 1.38e+00 3.32e+12  -1.0 4.00e+03    -  3.45e-01 5.01e-04h 10
 368  0.0000000e+00 1.38e+00 3.32e+12  -1.0 4.00e+03    -  1.88e-02 4.69e-04h 10
 369  0.0000000e+00 1.38e+00 3.32e+12  -1.0 3.99e+03    -  1.00e+00 4.42e-04h 10
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 370  0.0000000e+00 1.38e+00 3.32e+12  -1.0 3.99e+03    -  1.63e-02 4.17e-04h 10
 371  0.0000000e+00 1.38e+00 3.32e+12  -1.0 3.98e+03    -  3.49e-01 3.96e-04h 10
 372  0.0000000e+00 1.38e+00 3.32e+12  -1.0 3.98e+03    -  1.70e-02 3.77e-04h 10
 373  0.0000000e+00 1.38e+00 3.32e+12  -1.0 3.98e+03    -  1.00e+00 3.61e-04h 10
 374  0.0000000e+00 1.38e+00 3.32e+12  -1.0 3.97e+03    -  1.50e-02 3.47e-04h 10
 375  0.0000000e+00 9.85e+00 2.89e+12  -1.0 3.97e+03    -  3.53e-01 1.71e-01w  1
 376  0.0000000e+00 9.75e+00 3.59e+12  -1.0 3.38e+03    -  1.00e+00 2.84e-02w  1
 377  0.0000000e+00 9.75e+00

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 460  0.0000000e+00 1.31e+00 3.67e+12  -1.0 3.27e+03    -  1.00e+00 4.89e-03h  8
 461  0.0000000e+00 1.31e+00 3.68e+12  -1.0 3.26e+03    -  6.29e-01 4.90e-03h  8
 462  0.0000000e+00 1.31e+00 3.69e+12  -1.0 3.24e+03    -  1.00e+00 4.91e-03h  8
 463  0.0000000e+00 1.30e+00 3.70e+12  -1.0 3.23e+03    -  6.60e-01 4.92e-03h  8
 464  0.0000000e+00 1.30e+00 3.71e+12  -1.0 3.21e+03    -  1.00e+00 4.93e-03h  8
 465  0.0000000e+00 1.30e+00 3.72e+12  -1.0 3.20e+03    -  6.92e-01 4.94e-03h  8
 466  0.0000000e+00 7.97e+01 1.43e+12  -1.0 3.18e+03    -  1.00e+00 6.33e-01w  1
 467  0.0000000e+00 6.67e+01 2.97e+12  -1.0 1.22e+03    -  8.75e-01 1.78e-01w  1
 468  0.0000000e+00 6.64e+01 4.18e+14  -1.0 1.00e+03    -  6.61e-01 5.19e-03w  1
 469  0.0000000e+00 1.30e+00 3.73e+12  -1.0 9.90e+02    -  1.00e+00 4.95e-03h  7
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 470  0.0000000e+00 1.31e+00

In [20]:
model.Q_main.value

0

In [21]:
model.T.value

473.15

In [24]:
# model.solutions.store_to(results)
# with open('../saved_solutions/reboiler_MPCC_P_pf_200C.pickle','wb') as f:
#     pickle.dump(results,f)

In [23]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}\t\t\tlog K'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}\t\t\t{:.4f}'.format(model.x[i].value,model.y[i].value,np.log10(model.y[i].value/model.x[i].value)))

Component		Liquid: 1.0000			Vapor: 0.0000			log K
------------------------------------------------------------------------------------------------------------
H2         		 0.8837%			25.2934%			1.4567
CO         		 1.3401%			28.6944%			1.3307
CO2        		 3.7245%			31.8624%			0.9322
H2O        		 3.6023%			12.4034%			0.5370
C2H4       		 0.0174%			 0.1174%			0.8302
C3H6       		 0.0444%			 0.1351%			0.4829
C4H8       		 0.0456%			 0.1204%			0.4221
C5H10      		 0.0722%			 0.0931%			0.1104
C6H12      		 0.1262%			 0.0865%			-0.1641
C7H14      		 0.2078%			 0.0799%			-0.4151
C8H16      		 0.3264%			 0.0731%			-0.6499
C9H18      		 0.4920%			 0.0659%			-0.8729
C10H20     		 0.7120%			 0.0583%			-1.0869
C11H22     		 0.6286%			 0.0320%			-1.2938
C12H24     		 0.8302%			 0.0265%			-1.4952
C13H26     		 1.0411%			 0.0212%			-1.6918
C14H28     		 1.2375%			 0.0161%			-1.8847
C15H30     		 1.3970%			 0.0118%			-2.0743
C16H32     		 1.5062%			 0.0083%			-2.2611
C17H34     		 1.5634%			 0.0056%

# Iterative Solve for Data Analysis

In [ ]:
update_dual(pe,model)

In [ ]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [ ]:
rf_data = {}
rf_data['T'] = []; rf_data['Q'] = []; rf_data['V'] = []; rf_data['L'] = []; 
rf_data['y_CO'] = []; rf_data['y_H2'] = []; rf_data['y_CO2'] = []; rf_data['y_H2O'] = [];
rf_data['f_V_CO'] = []; rf_data['f_V_H2'] = [];
    
Trange = np.arange(350+273.15,199+273.15,-5)

for Tub in Trange:
    model.T.setub(Tub)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    # print('-'*72)
    print('Solved, Solution T = {:.2f} K\t|\tV = {:.5f}\t|\tL = {:.5f}\t|'.format(model.T.value,model.V['out'].value,model.L['out'].value))
    # print('-'*72)
    rf_data['T'].append(model.T.value)
    rf_data['Q'].append(model.Q_main.value)
    rf_data['V'].append(model.V['out'].value)
    rf_data['L'].append(model.L['out'].value)    
    
    
    rf_data['y_H2O'].append(model.y['H2O'].value)
    rf_data['y_CO'].append(model.y['CO'].value)
    rf_data['y_H2'].append(model.y['H2'].value)
    rf_data['y_CO2'].append(model.y['CO2'].value)
    
    rf_data['f_V_CO'].append(model.f_V['CO'].value)
    rf_data['f_V_H2'].append(model.f_V['H2'].value)

# VLE Validation using AspenPlus

### Feed (Reactor Effluent)

In [ ]:
print('Temperature (C): \t{:.2f}\t\t'.format(model.T.value-273.15))
print('Total FLow (kmol/s): \t{:.2f}\t\t'.format(model.L['out'].value+model.V['out'].value))
print('-'*72)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:.4%}'.format((model.x[i].value*model.L['out'].value\
        +model.y[i].value*model.V['out'].value)/(model.L['out'].value+model.V['out'].value)))

### Phase Separation

In [ ]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}\t\t\tlog K'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}\t\t\t{:.4f}'.format(model.x[i].value,model.y[i].value,np.log10(model.y[i].value/model.x[i].value)))

In [ ]:
list(zip(rf_data['T'],rf_data['Q']))